In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
np.random.seed(0)

In [2]:
df = pd.read_csv('Book1.csv')
print(df)
df.head()

    Sno  T_data_1_1  T_data_1_2  T_data_1_3  T_data_2_1  T_data_2_2  \
0     1         212         210         211         347         353   
1     2         212         211         211         346         352   
2     3         212         211         211         345         352   
3     4         213         211         211         344         351   
4     5         213         211         211         343         350   
..  ...         ...         ...         ...         ...         ...   
94   95         209         204         213         287         260   
95   96         210         204         213         287         260   
96   97         210         203         213         287         259   
97   98         210         203         213         287         258   
98   99         210         203         213         286         258   

    T_data_2_3  T_data_3_1  T_data_3_2  T_data_3_3  T_data_4_1  T_data_4_2  \
0          347         474         473         481         346       

,Sno,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
0,1,212,210,211,347,353,347,474,473,481,346,348,355,241,241,243,167.85,9.22
1,2,212,211,211,346,352,346,475,473,481,349,348,355,241,241,243,162.51,9.22
2,3,212,211,211,345,352,346,476,473,481,352,349,355,242,241,242,164.99,9.22
3,4,213,211,211,344,351,346,477,473,481,355,349,355,242,241,242,167.34,9.22
4,5,213,211,211,343,350,346,478,473,482,358,349,355,243,241,242,163.04,9.22


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error, explained_variance_score



X = df.drop(['Sno', 'H_data', 'AH_data'], axis=1)
y = df['AH_data']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

# Print the evaluation metrics
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('R-squared:', r2)
print('Mean Absolute Error:', mae)
print('Explained Variance Score:', evs)


Mean Squared Error: 7.839999999998522e-05
Root Mean Squared Error: 0.008854377448470627
R-squared: 0.9998333333333334
Mean Absolute Error: 0.002800000000015146
Explained Variance Score: 0.9998333333333334


In [4]:
import pandas as pd

materials_df = pd.read_csv('book1.csv') 

interactions_df = pd.read_csv('book1.csv') 

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
scaler = StandardScaler()
materials_scaled = scaler.fit_transform(materials_df.drop('Sno', axis=1))
material_similarity = cosine_similarity(materials_scaled)

In [6]:
material_similarity_df = pd.DataFrame(material_similarity, index=materials_df['Sno'], columns=materials_df['Sno'])
def get_content_based_recommendations(material_id, top_n=5):
    similar_materials = material_similarity_df[material_id].sort_values(ascending=False).head(top_n + 1).index.tolist()
    similar_materials.remove(material_id)
    return similar_materials

In [7]:
material_similarity = cosine_similarity(materials_scaled)
material_similarity_df = pd.DataFrame(material_similarity, index=materials_df['Sno'], columns=materials_df['Sno'])
def get_content_based_recommendations(material_id, top_n=5):
    similar_materials = material_similarity_df[material_id].sort_values(ascending=False).head(top_n + 1).index.tolist()
    similar_materials.remove(material_id)
    return similar_materials
print(get_content_based_recommendations(material_id=1))

[4, 3, 7, 6, 8]


In [8]:

import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy

# Load the user interactions dataset
interactions_df = pd.read_csv('book1.csv')  # This should contain user ratings/interactions
# Example columns: ['user_id', 'material_id', 'rating']

# Convert the interaction data to the format required by surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(interactions_df[['Sno', 'H_data', 'AH_data']], reader)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# Define the SVD algorithm
algo = SVD()

# Train the recommendation model using the training data
algo.fit(trainset)

# Evaluate the performance on the test set
predictions = algo.test(testset)
print(f'RMSE: {accuracy.rmse(predictions)}')

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_epochs': [20, 30],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

# Best parameters
print("Best RMSE score obtained: ", gs.best_score['rmse'])
print("Best parameters: ", gs.best_params['rmse'])

# Train the final model with the best parameters
best_algo = gs.best_estimator['rmse']
best_algo.fit(trainset)

# Evaluate the final model on the test set
final_predictions = best_algo.test(testset)
print(f'Final RMSE: {accuracy.rmse(final_predictions)}')


RMSE: 3.6569
RMSE: 3.656938610367968
Best RMSE score obtained:  3.7311473900724397
Best parameters:  {'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.4}
RMSE: 3.6569
Final RMSE: 3.656938610367968


In [9]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy
from collections import defaultdict


In [10]:
algo = SVD()
algo.fit(trainset)

predictions = algo.test(testset)
print(f'Initial RMSE: {accuracy.rmse(predictions)}')

param_grid = {
    'n_epochs': [20, 30],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print("Best RMSE score obtained: ", gs.best_score['rmse'])
print("Best parameters: ", gs.best_params['rmse'])

best_algo = gs.best_estimator['rmse']
best_algo.fit(trainset)

final_predictions = best_algo.test(testset)
print(f'Final RMSE: {accuracy.rmse(final_predictions)}')
print(f'Final MAE: {accuracy.mae(final_predictions)}')


RMSE: 3.6569
Initial RMSE: 3.656938610367968
Best RMSE score obtained:  3.7259464212069786
Best parameters:  {'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.4}
RMSE: 3.6569
Final RMSE: 3.656938610367968
MAE:  3.5900
Final MAE: 3.5900000000000007
